In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datasets import GeonamesDataset
import polars as pl

In [3]:
geonames = GeonamesDataset("./data/cities500.txt.gz", max_len=14)

In [4]:
geonames.df.sample(10)

sequence,feature code,country code,population
str,str,str,i64
"""Oefatu""","""PPLA4""","""ID""",0
"""Ballesteros""","""PPLA3""","""PH""",5591
"""Korlingen""","""PPLA4""","""DE""",788
"""Kushchyovskaya""","""PPLA2""","""RU""",29915
"""Mendlesham""","""PPL""","""GB""",884
"""Empersdorf""","""PPLA3""","""AT""",499
"""Witkowo""","""PPLA3""","""PL""",7825
"""Atapan""","""PPL""","""MX""",2323
"""Georgenberg""","""PPLA4""","""DE""",1507


In [5]:
df = geonames.df
alphabet = "".join(
    set("".join(df.get_column("sequence").str.split("").explode().to_list()))
)

In [6]:
from utils import Tokenizer

t = Tokenizer(
    alphabet=alphabet,
    max_len=16,
)

In [7]:
X = t.encode(df)

In [8]:
import torch

total_samples = X.size(0)

# Define the proportions for train, test, and validation sets
train_ratio = 0.8
test_ratio = 0.1
val_ratio = 0.1

# Calculate the number of samples for each set
num_train = int(total_samples * train_ratio)
num_test = int(total_samples * test_ratio)
num_val = total_samples - num_train - num_test

# Generate random indices
indices = torch.randperm(total_samples)

# Split the indices into train, test, and validation sets
train_indices = indices[:num_train]
test_indices = indices[num_train : num_train + num_test]
val_indices = indices[num_train + num_test :]

# Create the train, test, and validation sets
X_train = X[train_indices]
X_test = X[test_indices]
X_val = X[val_indices]

print("Train set size:", X_train.shape)
print("Test set size:", X_test.shape)
print("Validation set size:", X_val.shape)

Train set size: torch.Size([101200, 16])
Test set size: torch.Size([12650, 16])
Validation set size: torch.Size([12651, 16])


In [9]:
from torch.utils.data import TensorDataset, DataLoader

train = TensorDataset(X_train)
test = TensorDataset(X_test)
val = TensorDataset(X_val)

In [10]:
for i, batch_ in enumerate(DataLoader(train, batch_size=64)):
    break

# Model initialization

In [15]:
max_T = 100
d_embed = 32
token_dim = n_tokens = len(t.stoi)
d_kq = 12
d_hidden = 16

In [16]:
from diffusion import ForwardDiffusionProcess, Scheduler

from model import TokenDenoiser

scheduler = Scheduler(T=max_T)
diffusion = ForwardDiffusionProcess(scheduler=scheduler)
model = TokenDenoiser(
    max_T=max_T,
    max_L=token_dim,
    d_embed=d_embed,
    d_kq=d_kq,
    d_hidden=d_hidden,
    n_blocks=3,
)

In [17]:
sum((p.numel() for p in model.parameters()))

2807

# Model training

In [18]:
import torch.nn.functional as F
from torch.optim import AdamW
from tqdm.notebook import tqdm
import torch.optim.lr_scheduler as lr_scheduler

batch_size = 128
num_epochs = 10
optimizer = AdamW(model.parameters(), lr=1.0)

# scheduler = lr_scheduler.ExponentialLR(
# optimizer,
# gamma=0.99,
# )

for epoch in tqdm(range(num_epochs)):
    # get into training mode
    model.train()
    losses = []
    for batch_num in range(X_train.shape[0] // batch_size):
        optimizer.zero_grad()

        # initialize proper `x`
        ix = torch.randint(0, X_train.shape[0], size=(batch_size,))
        batch = X_train[ix].long()

        # encode into probas
        x_0 = F.one_hot(batch, num_classes=n_tokens).float()
        time = torch.randint(low=0, high=max_T, size=(x_0.shape[0],))

        x_t = diffusion.sample_T(x_0, time)
        x_0_pred = model(batch, time)

        loss = F.cross_entropy(x_0_pred, x_0)
        losses.append(loss.item())
        loss.backward()

    optimizer.step()
    # scheduler.step()

    # validation mode
    model.eval()
    with torch.no_grad():
        time = torch.randint(0, max_T, size=(X_test.shape[0],))
        batch_test = X_test.long()

        x_0_test = F.one_hot(batch_test, num_classes=n_tokens).float()
        x_t_test = diffusion.sample_T(x_0_test, time)
        x_0_test_pred = model(batch_test, time)
        test_loss = F.cross_entropy(x_0_test_pred, x_0_test)

        print(
            f"{epoch=}, train_loss={torch.tensor(losses).mean().item():2.4f}, val_loss={test_loss.item():2.4f}"
        )

        for p, tensor in model.named_parameters():
            print(f"\t{p=}, {tensor.grad.sum().item()=}")

  0%|          | 0/10 [00:00<?, ?it/s]

epoch=0, train_loss=0.8066, val_loss=0.8066
	p='blocks.0.weight', tensor.grad.sum().item()=0.00018581384210847318
	p='blocks.0.bias', tensor.grad.sum().item()=1.0211981134489179e-05
	p='blocks.1.Q', tensor.grad.sum().item()=-1.4984242397986236e-06
	p='blocks.1.K', tensor.grad.sum().item()=-4.107204176762025e-07
	p='blocks.1.V', tensor.grad.sum().item()=1.5006662579253316e-11
	p='blocks.1.norm.weight', tensor.grad.sum().item()=3.309003488993767e-07
	p='blocks.1.norm.bias', tensor.grad.sum().item()=-1.1856158721457177e-07
	p='blocks.2.Q', tensor.grad.sum().item()=1.3386625141720288e-11
	p='blocks.2.K', tensor.grad.sum().item()=1.3358203432289883e-12
	p='blocks.2.V', tensor.grad.sum().item()=-1.7905676941154525e-12
	p='blocks.2.norm.weight', tensor.grad.sum().item()=1.6777232758613536e-07
	p='blocks.2.norm.bias', tensor.grad.sum().item()=-2.134257215402613e-10
	p='blocks.3.weight', tensor.grad.sum().item()=-2.4374458007514477e-09
	p='blocks.3.bias', tensor.grad.sum().item()=2.091837814077

KeyboardInterrupt: 